In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-23 10:14:26--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8001::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-23 10:14:26 (9.16 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import json
import minsearch

In [90]:
with open ('/Users/dmitrywer/Desktop/my_projects/LLM_Zoomcamp/documents.json','r') as file:
    docs_raw = json.load(file)

In [91]:
len(file_input)

3

In [92]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [93]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [94]:
index = minsearch.Index(
    text_fields = ['question', 'text', 'section'],
    keyword_fields = ['course'])

In [95]:
index.fit(documents)

In [96]:
q = 'can i still enroll to the course?'

In [97]:
boost_dict = {'question' : 3, 'section' : 0.3}

index.search(q,
            filter_dict= {'course' : 'data-engineering-zoomcamp'},
            num_results= 1,
            boost_dict = boost_dict)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'}]

In [126]:
q = 'the course is already started, can I still join?'

In [127]:
from openai import OpenAI

In [128]:
client = OpenAI()

In [129]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

'Typically, many courses allow you to join even after they have started, though the specifics can depend on the institution, course structure, and how far along the course is. Here are a few steps you can take to find out if you can still join:\n\n1. **Check the Course Website or Portal**: Sometimes, the course details or FAQs will have information regarding late enrollments.\n\n2. **Contact the Instructor**: Email or message the course instructor directly. They can give you the most accurate information and may be willing to accommodate a late enrollment if you explain your situation.\n\n3. **Contact the Admissions or Registrar’s Office**: They can provide information on whether late enrollment is possible and what steps you need to take.\n\n4. **Consider Self-study or Auditing**: If joining officially late is not possible, you might ask if you can participate informally by auditing the course or catching up on your own with the materials provided.\n\nRemember, catching up might requi

In [130]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [125]:
context = ""
    
for doc in documents:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

prompt = prompt_template.format(question=q, context=context).strip()

In [117]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: the course is already started, can I still join?

CONTEXT: 
section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.

section: General course-related questions
question: Course - What are the prerequisites for this course?
answer: GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites

section: General course-relat

In [118]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{'role' : 'user', 'content' : prompt}])

In [119]:
response.choices[0].message.content

"Yes, you can still join the course even after it has started. According to the FAQ, you are eligible to join the course and submit homework even without prior registration. However, keep in mind that there are deadlines for submitting the final projects, so it's advisable to not leave everything for the last minute."